In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sncosmo

# Setup paths
project_root = Path("/Users/david/Code/msc/project")
data_path = project_root / "data"
src_path = project_root / "src"
sys.path.append(str(src_path))

print(f"sncosmo version: {sncosmo.__version__}")

sncosmo version: 2.12.1


In [ ]:
# Load light curve data
ztf_id = "ZTF17aabtvsy"
lasair_csv_path = data_path / "lasair" / f"{ztf_id}_lightcurve.csv"

if not lasair_csv_path.exists():
    print(f"Error: {lasair_csv_path} not found. Run the download script first.")
else:
    df = pd.read_csv(lasair_csv_path)
    print(f"Loaded {len(df)} data points for {ztf_id}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())

In [ ]:
# Prepare data for sncosmo
# sncosmo expects: time, band, flux, fluxerr (or mag, magerr with zp and zpsys)
# Our data has: MJD, filter, unforced_mag, unforced_mag_error

# Clean the data
df_clean = df.dropna(subset=['MJD', 'unforced_mag', 'filter']).copy()

# Convert filter names to sncosmo band names
# ZTF uses 'g' and 'r', sncosmo uses 'ztfg' and 'ztfr'
filter_map = {'g': 'ztfg', 'r': 'ztfr'}
df_clean['band'] = df_clean['filter'].map(filter_map)

# Drop rows with unmapped filters
df_clean = df_clean.dropna(subset=['band'])

# Convert to numeric
df_clean['mag'] = pd.to_numeric(df_clean['unforced_mag'], errors='coerce')
df_clean['magerr'] = pd.to_numeric(df_clean['unforced_mag_error'], errors='coerce')

# Fill NaN errors with a default value (e.g., 0.1 mag)
df_clean['magerr'] = df_clean['magerr'].fillna(0.1)

# Create sncosmo data table with magnitude format
# sncosmo can work with magnitude data if we provide zp and zpsys
zp = 25.0  # ZTF zero point magnitude
data = pd.DataFrame({
    'time': df_clean['MJD'].values,
    'band': df_clean['band'].values,
    'magnitude': df_clean['mag'].values,
    'magerr': df_clean['magerr'].values,
    'zp': zp,
    'zpsys': 'ab'
})

print(f"Prepared {len(data)} data points for fitting")
print(f"\nBands: {data['band'].unique()}")
print(f"\nTime range: {data['time'].min():.2f} to {data['time'].max():.2f} MJD")
print(f"\nMagnitude range: {data['magnitude'].min():.2f} to {data['magnitude'].max():.2f}")

In [ ]:
# Set up SALT2 model (Type Ia supernova model)
# Check available models
print("Available models:")
for model_name in sncosmo.builtins.list_source_names():
    print(f"  - {model_name}")

# Create SALT2 model
model = sncosmo.Model(source='salt2')

# Set initial parameters
# t0: time of maximum brightness (set to middle of observation range)
t0_guess = data['time'].min() + (data['time'].max() - data['time'].min()) / 2
model.set(t0=t0_guess)

# z: redshift (you may need to get this from metadata or fit it)
# For now, let's try fitting with redshift as a parameter
# Or use a known redshift if available
z_guess = 0.07  # Example redshift - adjust based on your data
model.set(z=z_guess)

print(f"\nModel parameters:")
print(f"  t0: {model.get('t0'):.2f} MJD")
print(f"  z: {model.get('z'):.4f}")
print(f"  x0: {model.get('x0'):.4e}")
print(f"  x1: {model.get('x1'):.2f}")
print(f"  c: {model.get('c'):.2f}")

In [ ]:
# Fit the model
# sncosmo.fit_lc can work with magnitude data if zp and zpsys are provided
# Or we can convert to flux first

# Method 1: Try with magnitude data directly
try:
    print("Attempting fit with magnitude data...")
    result, fitted_model = sncosmo.fit_lc(
        data,
        model,
        ['t0', 'x0', 'x1', 'c'],  # Parameters to fit
        bounds={'t0': (data['time'].min() - 20, data['time'].max() + 20),
                'x1': (-3.0, 3.0),
                'c': (-0.3, 0.3)},
        method='minuit'  # or 'emcee' for MCMC
    )
    
    print("✓ Fitting successful with magnitude data!")
    print(f"\nFitted parameters:")
    for param in ['t0', 'x0', 'x1', 'c']:
        value = fitted_model.get(param)
        print(f"  {param}: {value:.6f}")
    
    print(f"\nFit statistics:")
    print(f"  Chi-squared: {result.chisq:.2f}")
    print(f"  Number of degrees of freedom: {result.ndof}")
    print(f"  Reduced chi-squared: {result.chisq / result.ndof:.2f}")
    
except Exception as e:
    print(f"✗ Magnitude fit failed: {e}")
    print("\nTrying alternative approach with flux conversion...")
    
    # Method 2: Convert to flux first
    # Convert magnitude to flux: flux = 10^(-0.4 * (mag - zp))
    zp = 25.0  # ZTF zero point
    data_flux = data.copy()
    data_flux['flux'] = 10**(-0.4 * (data['magnitude'] - zp))
    # Flux error from magnitude error: fluxerr = flux * 0.4 * ln(10) * magerr
    data_flux['fluxerr'] = data_flux['flux'] * 0.4 * np.log(10) * data['magerr']
    
    # Remove magnitude columns for flux-based fit
    data_flux = data_flux[['time', 'band', 'flux', 'fluxerr']]
    
    try:
        result, fitted_model = sncosmo.fit_lc(
            data_flux,
            model,
            ['t0', 'x0', 'x1', 'c'],
            bounds={'t0': (data['time'].min() - 20, data['time'].max() + 20),
                    'x1': (-3.0, 3.0),
                    'c': (-0.3, 0.3)},
            method='minuit'
        )
        
        print("✓ Fitting successful with flux conversion!")
        print(f"\nFitted parameters:")
        for param in ['t0', 'x0', 'x1', 'c']:
            value = fitted_model.get(param)
            print(f"  {param}: {value:.6f}")
        
        print(f"\nFit statistics:")
        print(f"  Chi-squared: {result.chisq:.2f}")
        print(f"  Number of degrees of freedom: {result.ndof}")
        print(f"  Reduced chi-squared: {result.chisq / result.ndof:.2f}")
            
    except Exception as e2:
        print(f"✗ Flux conversion approach also failed: {e2}")
        print("\nNote: You may need to install 'iminuit' for the 'minuit' method,")
        print("      or try method='emcee' for MCMC fitting (requires 'emcee' package).")

In [ ]:
# Plot the fitted light curve
if 'fitted_model' in locals():
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot observed data
    for band in data['band'].unique():
        band_data = data[data['band'] == band]
        color = 'green' if 'g' in band else 'red'
        ax.errorbar(
            band_data['time'],
            band_data['magnitude'],
            yerr=band_data['magerr'],
            fmt='o',
            label=f'Observed {band}',
            color=color,
            alpha=0.7,
            markersize=4,
            capsize=2
        )
    
    # Plot model
    time_range = np.linspace(data['time'].min() - 10, data['time'].max() + 10, 200)
    for band in data['band'].unique():
        color = 'green' if 'g' in band else 'red'
        model_mag = fitted_model.bandmag(band, 'ab', time_range)
        ax.plot(time_range, model_mag, '--', color=color, alpha=0.8, linewidth=2, 
                label=f'Model {band}')
    
    ax.set_xlabel('MJD')
    ax.set_ylabel('Magnitude')
    ax.set_title(f'Light Curve Fit: {ztf_id}')
    ax.invert_yaxis()
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
else:
    print("No fitted model available to plot.")